# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
city_data = pd.read_csv("output_data/city_data.csv")
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Anadyr,64.75,177.48,44.26,93,90,15.66
1,1,Punta Arenas,-53.15,-70.92,23.00,92,20,4.70
2,2,Bandiagara,14.35,-3.61,92.59,43,91,11.52
3,3,Sítio Novo do Tocantins,-5.52,-47.46,87.80,45,81,14.99
4,4,Greymouth,-42.47,171.20,49.17,90,100,9.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Store Latitude and Longitude in Locations
locations = city_data[["Latitude", "Longitude"]].astype(float)

#Store Humidity in Humidity
humidity = city_data["Humidity"].astype(float)   

In [38]:
#Plot Heatmap
fig = gmaps.figure()

#Create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
#Add Heat Layer
fig.add_layer(heat_layer)

#Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
#Narrow down the cities to fit weather conditions.
ideal_cities = city_data.loc[(city_data["Temperature"] > 70) & (city_data["Temperature"] < 80) & (city_data["Cloudiness"] == 0) & (city_data["Wind Speed"] < 10), :]
ideal_cities.reset_index(inplace=True)
del ideal_cities['index']

#Drop any rows will null values.
ideal_cities = ideal_cities.dropna(how='any')
ideal_cities

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,122,Albany,42.60,-73.97,75.79,44,0,4.05
1,182,Lokachi,50.74,24.65,76.62,64,0,7.34
2,208,Henties Bay,-22.12,14.28,77.49,23,0,9.60
3,218,Chirilagua,13.22,-88.14,75.20,100,0,2.30
4,256,Huambo,-12.78,15.74,75.20,27,0,8.05
5,390,Harvard,42.42,-88.61,79.27,75,0,2.26
6,400,Sefophe,-22.18,27.97,75.20,73,0,4.70
7,421,Menongue,-14.66,17.69,79.59,17,0,9.84
8,465,Kriel,-26.27,29.23,70.50,16,0,3.00
9,467,Todos Santos,23.45,-110.22,73.49,88,0,4.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#Create list to store hotel names
hotellist = []

for i in range(len(ideal_cities)):
    lat = ideal_cities.loc[i]['Latitude']
    lng = ideal_cities.loc[i]['Longitude']
    
#Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
#Hit the Google Places API for each city's coordinates.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    
#Store the first Hotel result in the list     
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")

#Update the dataframe with Hotel Name column and populate from list       
ideal_cities["Hotel Name"] = hotellist

ideal_cities

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,122,Albany,42.60,-73.97,75.79,44,0,4.05,Voorheesville
1,182,Lokachi,50.74,24.65,76.62,64,0,7.34,Lokachi
2,208,Henties Bay,-22.12,14.28,77.49,23,0,9.60,Hentiesbaai
3,218,Chirilagua,13.22,-88.14,75.20,100,0,2.30,Chirilagua
4,256,Huambo,-12.78,15.74,75.20,27,0,8.05,Huambo
5,390,Harvard,42.42,-88.61,79.27,75,0,2.26,Harvard
6,400,Sefophe,-22.18,27.97,75.20,73,0,4.70,Sefhophe
7,421,Menongue,-14.66,17.69,79.59,17,0,9.84,Menongue
8,465,Kriel,-26.27,29.23,70.50,16,0,3.00,Kriel
9,467,Todos Santos,23.45,-110.22,73.49,88,0,4.56,Centro


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
locations_list = ideal_cities[["Latitude", "Longitude"]]

In [46]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations_list, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))